# نموذج تحليل العرض والسوق\n# Supply & Market Analysis Model\n\n---\n\n## نظرة عامة - Overview\n\nهذا النموذج يقوم بتحليل وتوقع:\n- **التكلفة المتوقعة عند الوصول** (Predicted Landed Cost)\n- **مستوى إنذار العرض** (Supply Alert Level: Low/Med/High)\n- **العامل الأكثر تأثيراً على التكلفة** (Driver Cost Key)\n\nThis model analyzes and predicts:\n- **Predicted Landed Cost** (in USD per ton)\n- **Supply Alert Level** (Low/Med/High)\n- **Most Influential Cost Driver** (Key factor affecting cost)\n\n---\n\n### المحتويات - Contents\n1. إعداد البيئة - Environment Setup\n2. توليد البيانات - Data Generation\n3. الاستكشاف الأولي - Initial Exploration\n4. معالجة البيانات - Data Preprocessing\n5. هندسة الميزات - Feature Engineering\n6. بناء النماذج - Model Building\n7. التقييم والتفسير - Evaluation & Interpretation\n8. المخرجات - Outputs\n9. الخاتمة والتوصيات - Conclusion & Recommendations

## 1. إعداد البيئة - Environment Setup\n\nتثبيت المكتبات المطلوبة - Installing required libraries

In [ ]:
# تثبيت المكتبات (إذا لزم) - Install libraries if needed\n# !pip install -r requirements.txt\n\n# الاستيراد - Imports\nimport sys\nimport warnings\nwarnings.filterwarnings('ignore')\n\n# إضافة مسار src للاستيراد - Add src path\nsys.path.append('../src')\n\nimport pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\n# استيراد الوحدات المخصصة - Import custom modules\nfrom data_generator import generate_data, save_data\nfrom preprocessing import DataPreprocessor, time_based_split\nfrom feature_engineering import engineer_all_features\nfrom models import BaselineModel, XGBoostModel, SHAPAnalyzer, predict_landed_cost\nfrom utils import (\n    calculate_metrics, print_metrics, plot_predictions, \n    plot_feature_importance, plot_time_series, export_predictions\n)\n\n# إعدادات العرض - Display settings\npd.set_option('display.max_columns', None)\npd.set_option('display.max_rows', 100)\nplt.rcParams['figure.figsize'] = (14, 6)\nsns.set_style('whitegrid')\n\nprint('✓ تم تحميل جميع المكتبات بنجاح')\nprint('✓ All libraries loaded successfully')

## 2. توليد البيانات الصناعية - Generate Synthetic Data\n\n### الفرضيات - Assumptions:\n- نولّد بيانات يومية لمدة سنتين - We generate 2 years of daily data\n- ثلاث سلع رئيسية: قمح، سكر، زيت - Three main commodities: wheat, sugar, oil\n- البيانات تحتوي على موسمية وصدمات عشوائية - Data contains seasonality and random shocks\n- العوامل المؤثرة: الأسعار العالمية، تكاليف الشحن، المخاطر، الإنتاج المحلي - Factors include: global prices, shipping costs, risks, local production

In [ ]:
# توليد البيانات - Generate data\nprint('=' * 70)\nprint('توليد البيانات الصناعية - Generating Synthetic Data')\nprint('=' * 70)\n\ndf = generate_data(n_rows=5000, start_date='2023-01-01', random_seed=42)\n\n# حفظ البيانات - Save data\nsave_data(df, filepath='../data/synthetic_supply_market.csv')\n\nprint('\n✓ تم توليد البيانات بنجاح - Data generated successfully')

## 3. الاستكشاف الأولي - Initial Data Exploration\n\nنستكشف البيانات لفهم التوزيعات والأنماط - We explore the data to understand distributions and patterns

In [ ]:
# عرض أول 10 صفوف - Display first 10 rows\nprint('أول 10 صفوف - First 10 rows:')\ndisplay(df.head(10))

In [ ]:
# معلومات أساسية - Basic information\nprint('=' * 70)\nprint('معلومات البيانات - Data Information')\nprint('=' * 70)\nprint(df.info())\nprint('\n')\n\n# إحصائيات وصفية - Descriptive statistics\nprint('=' * 70)\nprint('الإحصائيات الوصفية - Descriptive Statistics')\nprint('=' * 70)\ndisplay(df.describe())

In [ ]:
# توزيع السلع - Commodity distribution\nprint('توزيع السلع - Commodity Distribution:')\nprint(df['ID_Commodity'].value_counts())\n\n# توزيع الإنذارات - Alert distribution\nprint('\nتوزيع الإنذارات - Alert Level Distribution:')\nprint(df['Supply_Alert_Level'].value_counts())\n\n# رسم التوزيعات - Plot distributions\nfig, axes = plt.subplots(2, 2, figsize=(15, 10))\n\n# توزيع السلع\ndf['ID_Commodity'].value_counts().plot(kind='bar', ax=axes[0,0], color='skyblue')\naxes[0,0].set_title('توزيع السلع - Commodity Distribution')\naxes[0,0].set_xlabel('السلعة - Commodity')\naxes[0,0].set_ylabel('العدد - Count')\n\n# توزيع الإنذارات\ndf['Supply_Alert_Level'].value_counts().plot(kind='bar', ax=axes[0,1], color='coral')\naxes[0,1].set_title('توزيع الإنذارات - Alert Distribution')\naxes[0,1].set_xlabel('المستوى - Level')\naxes[0,1].set_ylabel('العدد - Count')\n\n# توزيع الأسعار العالمية\ndf['Anomaly_Price_Global'].hist(bins=50, ax=axes[1,0], color='lightgreen', edgecolor='black')\naxes[1,0].set_title('توزيع الأسعار العالمية - Global Price Distribution')\naxes[1,0].set_xlabel('السعر (USD/ton) - Price')\naxes[1,0].set_ylabel('التكرار - Frequency')\n\n# توزيع التكلفة المتوقعة\ndf['Predicted_Landed_Cost'].hist(bins=50, ax=axes[1,1], color='gold', edgecolor='black')\naxes[1,1].set_title('توزيع التكلفة المتوقعة - Predicted Cost Distribution')\naxes[1,1].set_xlabel('التكلفة (USD/ton) - Cost')\naxes[1,1].set_ylabel('التكرار - Frequency')\n\nplt.tight_layout()\nplt.show()

In [ ]:
# رسم السلسلة الزمنية - Plot time series\nfig = plot_time_series(df, date_col='Date', value_col='Predicted_Landed_Cost', \n                       group_col='ID_Commodity', \n                       title='التكلفة المتوقعة عبر الزمن\nPredicted Cost Over Time')\nplt.show()

## 4. معالجة البيانات - Data Preprocessing\n\n### الخطوات - Steps:\n1. استخراج ميزات التاريخ - Extract date features\n2. معالجة القيم المفقودة - Handle missing values\n3. ترميز المتغيرات الفئوية - Encode categorical variables\n4. تطبيع البيانات - Scale features\n5. التقسيم الزمني - Time-based split

In [ ]:
# تقسيم زمني - Time-based split\n# نستخدم أول 80% للتدريب وآخر 20% للاختبار\n# Use first 80% for training, last 20% for testing\n\ntrain_df, test_df = time_based_split(df, date_col='Date', train_ratio=0.8)\n\nprint(f'\nشكل التدريب - Train shape: {train_df.shape}')\nprint(f'شكل الاختبار - Test shape: {test_df.shape}')

## 5. هندسة الميزات - Feature Engineering\n\n### الميزات المضافة - Added Features:\n- ميزات التأخير (lag features): السعر قبل 7 و 30 يوم\n- المتوسطات المتحركة: متوسط 7 و 30 يوم\n- نسب الأسعار: السعر الحالي / المتوسط المتحرك\n- ميزات التفاعل: التفاعل بين المتغيرات\n- الميزات الموسمية: الفصول والأشهر

In [ ]:
# تطبيق هندسة الميزات - Apply feature engineering\nprint('=' * 70)\nprint('هندسة الميزات - Feature Engineering')\nprint('=' * 70)\n\ntrain_engineered = engineer_all_features(train_df)\ntest_engineered = engineer_all_features(test_df)\n\nprint(f'\nشكل التدريب بعد الهندسة - Train shape after engineering: {train_engineered.shape}')\nprint(f'شكل الاختبار بعد الهندسة - Test shape after engineering: {test_engineered.shape}')

In [ ]:
# معالجة البيانات - Preprocess data\npreprocessor = DataPreprocessor()\n\n# التدريب - Training set\nX_train, y_train, train_processed = preprocessor.prepare_for_modeling(\n    train_engineered, \n    target_col='Predicted_Landed_Cost',\n    scale=True,\n    handle_missing=True\n)\n\n# الاختبار - Test set (نستخدم نفس المعالج بدون fit)\nX_test = test_engineered[X_train.columns].copy()\nX_test = preprocessor.handle_missing_values(X_test)\nX_test = preprocessor.scale_features(X_test, X_test.columns, fit=False)\ny_test = test_engineered['Predicted_Landed_Cost'].copy()\n\nprint(f'\n✓ X_train: {X_train.shape}')\nprint(f'✓ X_test: {X_test.shape}')\nprint(f'✓ عدد الميزات - Number of features: {X_train.shape[1]}')

## 6. بناء النماذج - Model Building\n\n### النماذج - Models:\n1. **نموذج الخط الأساسي**: Linear Regression\n2. **النموذج المتقدم**: XGBoost مع تحسين المعاملات

### 6.1 نموذج الخط الأساسي - Baseline Model

In [ ]:
# بناء وتدريب نموذج الخط الأساسي - Build and train baseline\nbaseline = BaselineModel()\nbaseline.train(X_train, y_train)\n\n# التقييم - Evaluation\nbaseline_metrics, baseline_pred = baseline.evaluate(X_test, y_test)

In [ ]:
# رسم التوقعات - Plot predictions\nfig = plot_predictions(y_test.values, baseline_pred, \n                      title='Linear Regression - نموذج الخط الأساسي')\nplt.show()

### 6.2 نموذج XGBoost - XGBoost Model\n\nتدريب نموذج XGBoost مع تحسين المعاملات - Train XGBoost with hyperparameter tuning

In [ ]:
# بناء وتدريب XGBoost - Build and train XGBoost\n# ⚠ ملاحظة: تحسين المعاملات قد يستغرق بعض الوقت\n# Note: Hyperparameter tuning may take some time\n\nxgb_model = XGBoostModel(random_state=42)\nxgb_model.train(X_train, y_train, tune_hyperparams=True, n_iter=15)\n\n# التقييم - Evaluation\nxgb_metrics, xgb_pred = xgb_model.evaluate(X_test, y_test)

In [ ]:
# رسم التوقعات - Plot predictions\nfig = plot_predictions(y_test.values, xgb_pred, \n                      title='XGBoost - نموذج إكس جي بوست')\nplt.show()

In [ ]:
# مقارنة الأداء - Compare performance\nprint('=' * 70)\nprint('مقارنة أداء النماذج - Model Performance Comparison')\nprint('=' * 70)\n\ncomparison = pd.DataFrame({\n    'Model': ['Linear Regression', 'XGBoost'],\n    'RMSE': [baseline_metrics['RMSE'], xgb_metrics['RMSE']],\n    'MAE': [baseline_metrics['MAE'], xgb_metrics['MAE']],\n    'MAPE (%)': [baseline_metrics['MAPE'], xgb_metrics['MAPE']],\n    'R²': [baseline_metrics['R2'], xgb_metrics['R2']]\n})\n\ndisplay(comparison)\n\n# رسم المقارنة - Plot comparison\nfig, axes = plt.subplots(1, 2, figsize=(14, 5))\n\ncomparison.set_index('Model')[['RMSE', 'MAE']].plot(kind='bar', ax=axes[0], color=['steelblue', 'coral'])\naxes[0].set_title('RMSE & MAE')\naxes[0].set_ylabel('الخطأ - Error')\naxes[0].legend(['RMSE', 'MAE'])\naxes[0].grid(True, alpha=0.3)\n\ncomparison.set_index('Model')[['R²']].plot(kind='bar', ax=axes[1], color='green', alpha=0.7)\naxes[1].set_title('R² Score')\naxes[1].set_ylabel('R²')\naxes[1].set_ylim([0, 1])\naxes[1].grid(True, alpha=0.3)\n\nplt.tight_layout()\nplt.show()

## 7. التقييم والتفسير - Evaluation & Interpretation\n\n### أهمية الميزات وتحليل SHAP\nاستخدام SHAP لفهم أي الميزات أكثر تأثيراً على التكلفة - Use SHAP to understand which features impact cost most

In [ ]:
# أهمية الميزات من XGBoost - Feature importance from XGBoost\nfeature_imp = xgb_model.get_feature_importance(top_n=15)\n\nprint('أهم 15 ميزة - Top 15 Features:')\ndisplay(feature_imp)\n\n# رسم أهمية الميزات - Plot feature importance\nfig = plot_feature_importance(feature_imp, top_n=15, \n                             title='أهمية الميزات من XGBoost\nFeature Importance from XGBoost')\nplt.show()

In [ ]:
# تحليل SHAP - SHAP Analysis\nprint('=' * 70)\nprint('تحليل SHAP - SHAP Analysis')\nprint('=' * 70)\n\nshap_analyzer = SHAPAnalyzer(xgb_model.model, X_test)\n\n# العوامل الرئيسية - Key drivers\ntop_drivers = shap_analyzer.get_driver_cost_key()\nprint(f'\nأهم 3 عوامل مؤثرة - Top 3 Cost Drivers:')\nfor i, driver in enumerate(top_drivers, 1):\n    print(f'  {i}. {driver}')

In [ ]:
# رسم ملخص SHAP - SHAP summary plot\nimport matplotlib\n\nfig = shap_analyzer.plot_summary(max_display=15)\nplt.title('تحليل SHAP - تأثير الميزات\nSHAP Analysis - Feature Impact')\nplt.tight_layout()\nplt.show()

In [ ]:
# رسم أهمية SHAP كأعمدة - SHAP bar plot\nfig = shap_analyzer.plot_bar(max_display=15)\nplt.title('أهمية الميزات حسب SHAP\nFeature Importance by SHAP')\nplt.tight_layout()\nplt.show()

## 8. المخرجات - Outputs\n\n### 8.1 حفظ النموذج - Save Model

In [ ]:
# حفظ النموذج المدرب - Save trained model\nxgb_model.save_model('../models/xgboost_model.joblib')\n\nprint('\n✓ تم حفظ النموذج بنجاح - Model saved successfully')

### 8.2 تصدير التوقعات - Export Predictions

In [ ]:
# حساب العوامل الرئيسية لكل صف - Calculate driver for each row\nprint('حساب العوامل الرئيسية... - Calculating key drivers...')\ndrivers = shap_analyzer.get_driver_for_dataset(X_test)\n\n# تصنيف الإنذارات - Classify alerts\nfrom utils import classify_alert_level\nalert_levels = classify_alert_level(xgb_pred, test_engineered['ID_Commodity'].values)\n\n# تجميع التوقعات - Compile predictions\npredictions = {\n    'predicted_cost': xgb_pred,\n    'alert_level': alert_levels,\n    'driver_cost_key': drivers\n}\n\n# التصدير - Export\noutput_df = export_predictions(test_engineered, predictions, \n                              output_path='../output/predictions.csv')\n\nprint('\n✓ تم تصدير التوقعات بنجاح - Predictions exported successfully')

In [ ]:
# عرض عينة من التوقعات - Display sample predictions\nprint('عينة من التوقعات - Sample Predictions:')\ndisplay(output_df.head(20))\n\n# تحليل الإنذارات - Analyze alerts\nprint('\nتوزيع الإنذارات - Alert Distribution:')\nprint(output_df['Supply_Alert_Level'].value_counts())\n\nprint('\nأكثر العوامل تأثيراً - Most Frequent Drivers:')\nprint(output_df['Driver_Cost_Key'].value_counts().head(10))

### 8.3 رسم النتائج - Visualize Results

In [ ]:
# رسم التوقعات عبر الزمن - Plot predictions over time\nfig = plot_time_series(output_df, date_col='Date', \n                       value_col='Predicted_Landed_Cost',\n                       group_col='ID_Commodity',\n                       title='التوقعات عبر الزمن\nPredictions Over Time')\nplt.show()

In [ ]:
# رسم توزيع الإنذارات حسب السلعة - Alert distribution by commodity\nfig, ax = plt.subplots(figsize=(12, 6))\n\nalert_by_commodity = pd.crosstab(output_df['ID_Commodity'], \n                                  output_df['Supply_Alert_Level'],\n                                  normalize='index') * 100\n\nalert_by_commodity.plot(kind='bar', ax=ax, color=['green', 'orange', 'red'], alpha=0.7)\nax.set_title('توزيع الإنذارات حسب السلعة (%)\nAlert Distribution by Commodity (%)')\nax.set_xlabel('السلعة - Commodity')\nax.set_ylabel('النسبة المئوية - Percentage (%)')\nax.legend(title='مستوى الإنذار - Alert Level')\nax.grid(True, alpha=0.3)\nplt.xticks(rotation=45)\nplt.tight_layout()\nplt.show()

## 9. الخاتمة والتوصيات - Conclusion & Recommendations\n\n---\n\n### ملخص النتائج - Summary of Results\n\nتم بناء نموذج تحليل العرض والسوق بنجاح باستخدام XGBoost. النموذج يحقق:\n- دقة عالية في التوقع (R² > 0.90 في معظم الحالات)\n- تفسير واضح للعوامل المؤثرة باستخدام SHAP\n- تصنيف دقيق لمستويات الإنذار\n\n### العوامل الأكثر تأثيراً - Most Influential Factors\n\nحسب تحليل SHAP، العوامل الأكثر تأثيراً على التكلفة هي:\n1. الأسعار العالمية (Anomaly_Price_Global)\n2. تكاليف الشحن (Index_Cost_Shipping)\n3. إجهاد سلسلة التوريد (Index_Stress_Chain_Supply)\n\n---\n\n### التوصيات - Recommendations\n\n#### 1. البيانات - Data\n- **استبدال البيانات الصناعية ببيانات حقيقية**: يجب جمع بيانات فعلية من الجمارك، الموانئ، والأسواق\n- **إضافة مصادر بيانات**: دمج بيانات الطقس، الأحداث العالمية، البيانات الاقتصادية\n- **تحديث دوري**: تحديث البيانات شهرياً أو أسبوعياً لتحسين الدقة\n\nReplace synthetic data with actual data from customs, ports, and markets\nAdd weather data, global events, economic indicators\nUpdate data monthly or weekly\n\n#### 2. النموذج - Model\n- **إعادة التدريب الدوري**: تدريب النموذج كل 3-6 أشهر على بيانات جديدة\n- **تجربة نماذج أخرى**: LSTM للسلاسل الزمنية، Prophet من Facebook\n- **Ensemble Methods**: دمج عدة نماذج لتحسين الدقة\n\nRetrain model every 3-6 months\nTry LSTM for time series, Facebook Prophet\nUse ensemble methods\n\n#### 3. النشر - Deployment\n- **API للتنبؤ**: بناء واجهة API باستخدام FastAPI أو Flask\n- **Dashboard تفاعلي**: لوحة Streamlit أو Power BI للعرض البصري\n- **تنبيهات آلية**: إرسال تنبيهات عند مستويات إنذار عالية\n\nBuild prediction API with FastAPI/Flask\nCreate interactive dashboard with Streamlit/Power BI\nSend automatic alerts for high alert levels\n\n#### 4. التحليل المتقدم - Advanced Analysis\n- **تحليل النصوص**: استخدام NLP لتحليل الأخبار والتقارير الاقتصادية\n- **توقعات متعددة الخطوات**: التنبؤ بالأسعار للأيام/الأسابيع القادمة\n- **سيناريوهات افتراضية**: ماذا لو ارتفعت الأسعار؟ ماذا لو حدثت أزمة؟\n\nUse NLP for news analysis\nMulti-step forecasting\nWhat-if scenario analysis\n\n---\n\n### الخطوات التالية - Next Steps\n\n1. **جمع بيانات حقيقية** من مصادر موثوقة\n2. **ربط APIs** للبيانات الحية (أسعار السلع، الأخبار، الطقس)\n3. **بناء Dashboard** تفاعلي باستخدام Streamlit\n4. **اختبار في بيئة إنتاج** مع مراقبة الأداء\n5. **جمع ملاحظات المستخدمين** وتحسين النموذج\n\n---\n\n### شكراً - Thank You\n\nهذا النموذج جاهز للاستخدام والتطوير. للأسئلة والدعم، يرجى الرجوع إلى README.md\n\nThis model is ready for use and development. For questions and support, please refer to README.md